In [5]:
import pandas
import json
import string
import ijson

In [105]:
class Dataset:
    def __init__(self):
        self.ReviewsLocation = "~/dataset_toobig_for_github/review.json"
        self.UnigramBagOfWords = {};
        self.BigramBagOfWords = {};
        self.StopWords = {'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'} #https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
        self.Symbols = string.punctuation
        self.StarCount = {'5':0, '4':0, '3':0, '2':0, '1':0, '0':0}

    def RemoveStopWords(self, reviewWordList):
        '''
        Removes all stop words from a given review word list
        Input:
            reviewWordList: List of preprocessed words in review
        Output:
            cleanReviewList: List of preprocessed words without stopwords
        '''
        cleanReviewList = []
        for word in reviewWordList:
            cleanWord = word.strip()
            if cleanWord not in self.StopWords:
                cleanReviewList.append(cleanWord);
        return cleanReviewList

    def RemoveSymbols(self, reviewString):
        '''
        Removes all symbols from a review string
        Input:
            reviewString: review in string format
        Output:
            reviewString: review without any symbols defined in string.punctuation
        '''
        for symbol in self.Symbols:
            reviewString = reviewString.replace(symbol, '')
        return reviewString

    def WeightedMean(self, probabilities):
        '''
        Calculates the weighted, normalized mean for a set of probabilities for a review rating
        Input:
            probabilities: List of each star and their current probabilities
        Output:
            weightedMean: Returns the highest prediction or weighted mean
        '''
        rankings = [0, 1, 2, 3, 4, 5]
        try:
            norm = [float(i)/sum(probabilities) for i in probabilities]
            weightedMean = [norm[i] * rankings[i] for i in range(0,6)]
        except:
            weightedMean = max(probabilities)
            return weightedMean
        return sum(weightedMean)        
        
        
    def AddUnigramBagOfWords(self, word, stars):
        '''
        Adds a given word and star rating to the unigram bag of words model
        Input:
            word: Word to enter into dictionary
            stars: Star rating for the review corresponding to the word
        '''
        if word not in self.UnigramBagOfWords:
            self.UnigramBagOfWords[word] = {'5':0,'4':0,'3':0,'2':0,'1':0,'0':0}
        self.UnigramBagOfWords[word][stars] += 1
        
    def AddBigramBagOfWords(self, word_one, word_two, stars):
        '''
        Adds a given set of two words and the corresponding star rating to the review to the bigram bag of words model
        Input:
            word_one: First word of set found in the review
            word_two: Second word of set found in the review
            stars: Star rating associated with the review where the words were gathered
        '''
        if word_one not in self.BigramBagOfWords:
            self.BigramBagOfWords[word_one] = {word_two: {'5':0,'4':0,'3':0,'2':0,'1':0,'0':0}}
        if word_two not in self.BigramBagOfWords[word_one]:
            self.BigramBagOfWords[word_one][word_two] = {'5':0,'4':0,'3':0,'2':0,'1':0,'0':0}
        self.BigramBagOfWords[word_one][word_two][stars] += 1

    def GetUnigramReviewBagOfWords(self,count,save=False):
        '''
        Iterates through count number of reviews, parses the stars and text from dataset, preprocesses data,
        and adds in the word and star rating into the unigram
        Input:
            count: Number of reviews to add into the unigram model
            save: Initialized as false, saves the unigram into a json file
        '''
        # Chunk size set to 1, reads in one line at a time
        ReviewsReader = pandas.read_json(self.ReviewsLocation, lines=True, chunksize=1)
        index = -1;
        for review in ReviewsReader:
            index += 1
            try:
                reviewStars = str(review.stars[index]);
                reviewText = review.text[index].strip();
            except:
                print(review.stars)
                print(review.text)
                break
            
            # Remove Symbols
            for symbol in self.Symbols:
                reviewText = reviewText.replace(symbol, '')
            
            # Preprocess text
            reviewText = reviewText.lower()
            reviewText = reviewText.split(' ')
            
            # Remove stop words and add to dict
            for word in reviewText:
                cleanWord = word.strip()
                if cleanWord not in self.StopWords:
                    self.AddUnigramBagOfWords(cleanWord, reviewStars)
                    
            if index >= count:
                break
        
        # Save bagofwords dict
        if save == True:
            with open('unigrambagofwords' + str(count) + '.json', 'w') as fp:
                json.dump(self.UnigramBagOfWords, fp)
                
    def GetBigramReviewBagOfWords(self,save=False):
        '''
        Iterates through count number of reviews, parses the stars and text from dataset, preprocesses data,
        and adds in the word and star rating into the bigram model
        Input:
            save: Initialized as false, saves the bigram into a json file
        '''
        ReviewsReader = pandas.read_json(self.ReviewsLocation, lines=True, chunksize=1)
        index = -1;
        for review in ReviewsReader:
            index += 1
            try:
                reviewStars = str(review.stars[index]);
                reviewText = review.text[index].strip();
            except:
                print(review.stars)
                print(review.text)
                break
                
            reviewText = self.RemoveSymbols(reviewText)
            reviewText = reviewText.lower()
            reviewTextList = reviewText.split(' ')
            
            cleanText = self.RemoveStopWords(reviewTextList)
            
            for i in range(0,len(cleanText)-1):
                self.AddBigramBagOfWords(cleanText[i], cleanText[i+1], reviewStars)
            
        if save == True:
            with open('bigrambagofwords.json', 'w') as fp:
                json.dump(self.BigramBagOfWords, fp)
            
    def LoadBigram(self):
        '''
        Loads in a previously built and saved bigram model from bigrambagofwords.json
        '''
        jsonfile = open('bigrambagofwords.json')
        jsonstr = jsonfile.read()
        self.BigramBagOfWords = json.loads(jsonstr)

    def LoadUnigram(self):
        '''
        Loads in a previously built and saved unigram model from unigrambagofwords.json
        '''
        jsonfile = open('dataset/unigrambagofwords10000RatingReviews.json')
        jsonstr = jsonfile.read()
        self.UnigramBagOfWords = json.loads(jsonstr)
    
    def LoadUnigramCount(self, count):
        '''
        Loads in the unigram model from unigrambagofwords[count].json, where count is the number of reviews
        the data was generated from
        Input:
            count: Size of unigram model to load. Used as input into load file
        '''
        jsonfile = open('unigrambagofwords' + str(count) + '.json')
        jsonstr = jsonfile.read()
        self.UnigramBagOfWords = json.loads(jsonstr)
        

    def UnigramReviewScore(self, review, weightedMean=False):
        '''
        Calculates the star rating of a review using the unigram model
        Input:
            review: Review to give star rating to
            weightedMean: Initialized as false, normalizes the star rating to give an average as opposed to 
                            just a star rating
        Output:
            Star prediction
        '''
        review = self.RemoveSymbols(review)
        review = review.lower()
        review = review.split(' ')
        cleanReview = self.RemoveStopWords(review)

        probabilities = [1, 1, 1, 1, 1, 1]
        for word in cleanReview:
            if word not in self.UnigramBagOfWords:
                continue
            # Calculate total reviews for word
            totalReviews = 0;

            for star in range(0,6):
                starString = str(star)
                totalReviews += self.UnigramBagOfWords[word][starString]
            for star in range(0,6):
                starString = str(star)
                if self.UnigramBagOfWords[word][starString] != 0:
                    try:
                        # Determines the probability of a word showing up in a star review compared to the total number
                        # of star reviews
                        probabilities[star] *= (float(self.UnigramBagOfWords[word][starString] / float(totalReviews)))
                        #probabilities[star] *= (float(self.UnigramBagOfWords[word][starString] / float(self.StarCount[starString])))
                    except:
                        probabilities[star] *= (float(1) / float(250000))
                else:
                    # Calculates the probability of a star where the word hasn't shown up before by taking 1/total amount of review ratings
                    probabilities[star] *= (float(1) / float(250000))

        # Applies the weighted mean if boolean is true, else returns the highest probability for the star rating
        if weightedMean == True:
            return self.WeightedMean(probabilities)
        else:
            return probabilities.index(max(probabilities))

    def BigramReviewScore(self, review):
        '''
        Calculates the star rating of a review using the bigram model
        Input:
            review: Review to give star rating to
        Output:
            Star prediction
        '''
        review = self.RemoveSymbols(review)
        review = review.lower()
        review = review.split(' ')
        cleanReview = self.RemoveStopWords(review)

        probabilities = [1, 1, 1, 1, 1, 1]
        for i in range(0,len(cleanReview)-1):
            word_one = cleanReview[i]
            word_two = cleanReview[i+1]
            
            # Skips the word if the first and second word combination does not show up in the bigram model
            if word_one not in self.BigramBagOfWords:
                continue
            else:
                if word_two not in self.BigramBagOfWords[word_one]:
                    continue;

            # Calculate total reviews for word
            totalReviews = 0;
            for stars in range(0,5):
                totalReviews += self.BigramBagOfWords[word_one][word_two][stars]
            for stars in range(0,5):
                if self.UnigramBagOfWords[word_one][word_two][stars] != 0:
                    # Calculates the probability of the star review over the total amount of reviews for any given word combination
                    probabilties[stars] *= (float(self.BigramBagOfWords[word_one][word_two][stars]) / float(totalReviews))
                else:
                    # If the word doesn't show up, takes a naive probability
                    probabilties[stars] *= (float(1)/float(totalReviews))
        
        # Returns the highest probability
        return probabilities.index(max(probabilities))


    def iJsonParser(self,word):
        '''
        Using iJson, iterates through the unigram model to find a given word and returns the star ratings for that word
        Input:
            word: Word to look for
        Output:
            returnDict: Dictionary containing the ratings for each star of the given word
        '''
        parser = ijson.parse(open('unigrambagofwords.json'))
        dataCollected = 0;
        returnDict = {}
        returnDict[word] = {'5':0,'4':0,'3':0,'2':0,'1':0,'0':0}

        for key, value, pair in parser:
            if word not in key:
                continue
            for star in range(0,6):
                if key==(word + '.' + str(star)):
                    returnDict[word][str(star)] = pair
                    dataCollected += 1
            if dataCollected == 6:
                break
        return returnDict
    
    
    def iJsonParserTwo(self, word_one, word_two):
        '''
        Using iJson, iterates through the biram model to find a given word and returns the star ratings for that word
        Input:
            word_one: First word in bigram
            word_two: Second word in bigram
        Output:
            returnDict: Dictionary containing the ratings for each star of the given word
        '''
        parser = ijson.parse(open('bigrambagofwords.json'))
        dataCollected = 0;
        returnDict = {}
        returnDict[word_one] = {word_two: {'5':0,'4':0,'3':0,'2':0,'1':0,'0':0}} 

        
        for key, value, pair in parser:
            if word_one not in key and word_two not in key:
                continue
            for star in range(0,6):
                if key==(word_one + '.' + word_two + '.' + str(star)):
                    returnDict[word_one][word_two][str(star)] = pair
                    dataCollected += 1
            if dataCollected == 6:
                break
        return returnDict
                
                
    def UnigramReviewScoreiJson(self, review, weightedMean=False):
        '''
        Calculates the review score using the unigram model and the iterative iJson parser
        Input:
            review: review to rate
            weightedMean: Initialized to False, if true, calculates the weighted mean as opposed to a star rating
        Output:
            Prediction
        '''
        review = self.RemoveSymbols(review)
        review = review.lower()
        review = review.split(' ')
        cleanReview = self.RemoveStopWords(review)

        probabilities = [1, 1, 1, 1, 1, 1]
        for word in cleanReview:
            unigram = self.iJsonParser(word)
            if word not in unigram:
                continue
            # Calculate total reviews for word
            totalReviews = 0;

            for star in range(0,6):
                starString = str(star)
                totalReviews += unigram[word][starString]
            for star in range(0,6):
                starString = str(star)
                if unigram[word][starString] != 0:
                    probabilities[star] *= (float(unigram[word][starString]) / float(totalReviews))
                else:
                    try:
                        probabilities[star] *= (float(1) / float(totalReviews))
                    except:
                        probabilities[star] *= (float(1)/float(1000000))

        if weightedMean == True:
            return self.WeightedMean(probabilities)
        else:
            return probabilities.index(max(probabilities))
        
        
    def BigramReviewScoreiJson(self, review, weightedMean=False):
        '''
        Calculates the review score using the bigram model and the iterative iJson parser
        Input:
            review: review to rate
            weightedMean: Initialized to False, if true, calculates the weighted mean as opposed to a star rating
        Output:
            Prediction
        '''
        review = self.RemoveSymbols(review)
        review = review.lower()
        review = review.split(' ')
        cleanReview = self.RemoveStopWords(review)

        probabilities = [1, 1, 1, 1, 1, 1]
        for i in range(0,len(cleanReview)-1):
            word_one = cleanReview[i]
            word_two = cleanReview[i+1]
            bigram = self.iJsonParserTwo(word_one, word_two)

            if word_one not in bigram:
                continue
            else:
                if word_two not in bigram[word_one]:
                    continue;

            # Calculate total reviews for word
            totalReviews = 0;
            for stars in range(0,6):
                stringStars = str(stars)
                totalReviews += bigram[word_one][word_two][stringStars]
            for stars in range(0,6):
                stringStars = str(stars)

                if bigram[word_one][word_two][stringStars] != 0:
                    probabilities[stars] *= (float(bigram[word_one][word_two][stringStars]) / float(totalReviews))
                else:
                    try:
                        probabilities[stars] *= (float(1)/float(totalReviews))
                    except:
                        probabilities[stars] *= (float(1)/float(1000000))
                        
        if weightedMean == True:
            return self.WeightedMean(probabilities)
        else:
            return probabilities.index(max(probabilities))

    def RankUnigramReviews(self,save=False):
        '''
        Ranks a set of reviews using the ijson model
        Input:
            save: Initialized as False, saves unigram predictions
        '''
        predictions = {}
        ReviewsReader = pandas.read_json(self.ReviewsLocation, lines=True, chunksize=1)
        index = -1;
        for review in ReviewsReader:
            index += 1
            try:
                reviewID = str(review.review_id[index]);
                reviewStars = str(review.stars[index]);
                reviewText = review.text[index].strip();
            except:
                print(review.stars)
                print(review.text)
                print(review.review_id)
                break
            
            prediction = self.UnigramReviewScoreiJson(reviewText, weightedMean=True)
            predictions[reviewID] = prediction
        
            if index > 50000:
                break
        
        # Save bagofwords dict
        if save == True:
            with open('unigramPredictions.json', 'w') as fp:
                json.dump(predictions, fp)
                
                
    def RankUnigramReviewsInRam(self,save=False):
        '''
        Ranks a set of reviews, 50000, by initially loading the unigram model into RAM
        Input:
            save: Initialized as False, saves unigram predictions
        '''
        predictions = {}
        countReviews = {'1':0,'2':0,'3':0,'4':0,'5':0}

        ReviewsReader = pandas.read_json(self.ReviewsLocation, lines=True, chunksize=1)
        index = -1;
        for review in ReviewsReader:
            index += 1
            try:
                reviewID = str(review.review_id[index]);
                reviewStars = str(review.stars[index]);
                reviewText = review.text[index].strip();
            except:
                print(review.stars)
                print(review.text)
                print(review.review_id)
                break
                
            if countReviews['1'] >= 10000 and countReviews['2'] >= 10000 and countReviews['3'] >= 10000 and countReviews['4'] >= 10000 and countReviews['5'] >= 10000:
                print(countReviews)
                break
            
            
            if countReviews[reviewStars] < 10000:
                countReviews[reviewStars] += 1
            else:
                continue
            
            prediction = self.UnigramReviewScore(reviewText, weightedMean=True)
            predictions[reviewID] = prediction
            print(str(reviewID) + ': ' + str(prediction))
            
            if index % 5000 == 0:
                print(countReviews)
            
        
        # Save bagofwords dict
        if save == True:
            with open('unigramPredictionsTest.json', 'w') as fp:
                json.dump(predictions, fp)
                
    def RankBigramReviews(self,save=False, weightedMean=False):
        '''
        Ranks a set of reviews, 15000, using the bigram model
        Input:
            save: Initialized as False, will save model if True to a JSON object
            weightedMean: Initialized as False, will calculate the weighted mean prediction if true
        '''
        predictions = {}
        ReviewsReader = pandas.read_json(self.ReviewsLocation, lines=True, chunksize=1)
        index = -1;
        for review in ReviewsReader:
            index += 1
            try:
                reviewID = str(review.review_id[index]);
                reviewStars = str(review.stars[index]);
                reviewText = review.text[index].strip();
            except:
                print(review.stars)
                print(review.text)
                print(review.review_id)
                break
            if index >= 15000:
                break
            prediction = self.BigramReviewScoreiJson(reviewText)
            predictions[reviewID] = prediction
        
        # Save bagofwords dict
        if save == True:
            with open('bigramPredictions.json', 'w') as fp:
                json.dump(predictions, fp)
                
    def CountStarRankings(self, count): 
        '''
        Calculates the amount of reviews of each star up to a count number of reviews. Saves model.
        Input:
            count: Number of reviews to count ratings of
        '''
        ReviewsReader = pandas.read_json(self.ReviewsLocation, lines=True, chunksize=1)
        index = -1;
        for review in ReviewsReader:
            index += 1
            try:
                reviewStars = str(review.stars[index]);
            except:
                break
            if index >= count:
                break
            self.StarCount[reviewStars] += 1
        
        # Save starCount dict
        total = 0;
        for stars in self.StarCount:
            total += self.StarCount[stars]
        self.StarCount['total'] = total
        with open('starCount' + str(count) + '.json', 'w') as fp:
            json.dump(self.StarCount, fp)
            
            
    def GetUnigramReviewBagOfWordsNumberRatingReviews(self,count,save=False):
        '''
        Iterates through count number of reviews, parses the stars and text from dataset, preprocesses data,
        and adds in the word and star rating into the unigram
        Input:
            count: Number of reviews to add into the unigram model
            save: Initialized as false, saves the unigram into a json file
        '''
        # Chunk size set to 1, reads in one line at a time
        ReviewsReader = pandas.read_json(self.ReviewsLocation, lines=True, chunksize=1)
        countReviews = {'1':0,'2':0,'3':0,'4':0,'5':0}
        index = -1;
        for review in ReviewsReader:
            index += 1
            try:
                reviewStars = str(review.stars[index]);
                reviewText = review.text[index].strip();
            except:
                print(review.stars)
                print(review.text)
                break
            
            # Remove Symbols
            for symbol in self.Symbols:
                reviewText = reviewText.replace(symbol, '')
            
            # Preprocess text
            reviewText = reviewText.lower()
            reviewText = reviewText.split(' ')
            
            if countReviews[reviewStars] < count:
                countReviews[reviewStars] += 1
            else:
                continue
            
            # Remove stop words and add to dict
            for word in reviewText:
                cleanWord = word.strip()
                if cleanWord not in self.StopWords:
                    self.AddUnigramBagOfWords(cleanWord, reviewStars)
        
        
            if index % 5000 == 0:
                print(countReviews)
            if countReviews['1'] >= count and countReviews['2'] >= count and countReviews['3'] >= count and countReviews['4'] >= count and countReviews['5'] >= count:
                print(countReviews)
                break
        
        # Save bagofwords dict
        if save == True:
            with open('unigrambagofwords' + str(count) + 'RatingReviews.json', 'w') as fp:
                json.dump(self.UnigramBagOfWords, fp)
        

In [106]:
dataset = Dataset()
#dataset.GetUnigramReviewBagOfWords(save=True)
#dataset.GetBigramReviewBagOfWords(save=True)

In [ ]:
dataset.GetUnigramReviewBagOfWordsNumberRatingReviews(10000,save=True)
dataset.LoadUnigram()
#dataset.CountStarRankings(50000)
#dataset.GetUnigramReviewBagOfWords(50000, save=True)
#dataset.RankBigramReviews(save=True, weightedMean = True)
#dataset.RankUnigramReviews(save=True)
#dataset.LoadUnigram()
#dataset.LoadBigram()

{'1': 0, '2': 0, '3': 0, '4': 0, '5': 1}
{'1': 393, '2': 446, '3': 939, '4': 1613, '5': 1610}
{'1': 873, '2': 891, '3': 1716, '4': 2966, '5': 3555}
{'1': 1399, '2': 1308, '3': 2385, '4': 4419, '5': 5490}
{'1': 2004, '2': 1742, '3': 3044, '4': 5746, '5': 7465}
{'1': 2737, '2': 2155, '3': 3618, '4': 6854, '5': 9637}
{'1': 7508, '2': 4924, '3': 7616, '4': 10000, '5': 10000}
{'1': 8202, '2': 5345, '3': 8121, '4': 10000, '5': 10000}
{'1': 9602, '2': 6214, '3': 9190, '4': 10000, '5': 10000}
{'1': 10000, '2': 9958, '3': 10000, '4': 10000, '5': 10000}
{'1': 10000, '2': 10000, '3': 10000, '4': 10000, '5': 10000}


In [ ]:
#dataset.LoadUnigramCount(50000)
dataset.RankUnigramReviewsInRam(save=True)

v0i_UHJMo_hPBq9bxWvW4w: 5.0
{'1': 0, '2': 0, '3': 0, '4': 0, '5': 1}
vkVSCC7xljjrAI4UGfnKEQ: 5.0
n6QzIUObkYshz4dz2QRJTw: 5.0
MV3CcKScW05u5LVfF6ok0g: 5.0
IXvOzsEMYtiJI0CARmj77Q: 4.0
L_9BTb55X0GDtThi6GlZ6w: 4.0
HRPm3vEZ_F-33TYVT7Pebw: 5.0
ymAUG8DZfQcFTBSOiaNN4w: 4.0
8UIishPUD92hXtScSga_gw: 4.0
w41ZS9shepfO3uEyhXEWuQ: 3.0
WF_QTN3p-thD74hqpp2j-Q: 5.0
PIsUSmvaUWB00qv5KTF1xA: 4.0
PdZ_uFjbbkjtm3SCY_KrZw: 3.0
x5oV6wm9_Pb1QQ6jkjDjwQ: 1.0
lsoSqIrrDbQvWpMvsSj2xw: 3.0
23eqwlZzCWZkADWfd9atZw: 5.0
FunI9om-aK5oMIIJmhMlZA: 4.0
FKu4iU62EmWT6GZXPJ2sgA: 1.0
xdu8nXrbNKeaywCX79KZSw: 2.9999999999999956
K7o5jDInfmX3cY5oH6ATNw: 2.999999999993457
WYDFJOBOl7cycd7gN-c_xA: 1.0
HSR2RLOifd0cvSNVqGXkMQ: 3.0
Kki2nwtP8U2qmWwRvPwLRA: 4.0
Q-mhDIKa3wJuWEx9uuciIA: 3.0
ypjtMQLKdAwKGRS-KU7oxA: 1.0000000002751703
vr9YpzkaVdDIauxiZYZhqA: 3.0
Ia-w-nR1FrlzsiuEiqqlbg: 3.000000000000001
y21Fom8a_SdAyC6I0v554w: 2.0
_ku1sDwkmQo2wIgWAaluZw: 4.999999999993591
Enuk_DJbK0JPmgbFU8ePKw: 3.0
iYlepMJ6j4whB8gxiZm6mw: 4.0
reeZj98t_X1DrZgQgFs

uNpJmvEVR20tJqFaeHc-_w: 3.0
pYGsm7p8qukZlGlLx4WMcg: 2.9999999999932707
1P4QQ8OEiixcvAoV0Rv-yQ: 5.0
4KvYXnRg_1ymaTDxJPhtDg: 1.0
EM8zGThMXbMX0vGxMMUwrA: 4.0
y1w7pV6_eqVhKM_f5gi-IQ: 2.0
uPDs-rsp9Hho3ZTKzk1MuQ: 3.0000000000000333
BP38bvpVNJlgJdUDMsnWSg: 4.999999999961879
XdgHTfBI3VP1Xtp714gHgA: 4.999999999999935
WxqXT1REh-Ty4TL8dC1vcg: 3.0
JACmr8B0p9EZ1xnDyyZd4A: 1.0
LoONINX7OIxy2J_fIOspJA: 3.0
fqbifTr1PdIsDjhE5Lf_OA: 3.004185307552235
qWEbm-YoFs77hgD3r_ZniQ: 3.0
jUvqecQBO6ZrNeVTQfvySg: 3.999999999999849
A-U5mQneUASIyUkkE-H3sw: 2.0
BISA8iul0K3ViupdqHAV3Q: 2.0
6yjEBShfZJWKQECKNCVrpg: 4.0
uhvn5HFTipP-hZQBgnYBYg: 3.0000000000000173
ne0ua0nM9N7hsxufAOQOHQ: 5.0
IQCNtO_Djm_qeNpGWF9EgQ: 2.0
Ku6svu57zH_TdUjYUFunPg: 3.0
8gnSouFOa8arYfY0D_b8fg: 3.0
2v0CLzq80wgpsrvBQzwGvQ: 2.0
AMP7GP_aZDQKdEpNBY0XfQ: 4.0
SyvB9pp9Na0sQevs26Dq3Q: 3.0
3PtuAHTyDA6KOBU7olAbOA: 2.0
ED0ZG6hKWMji631ouEGWIA: 3.000000000000213
wqqWvE14aZEszf9Rr9pFNw: 4.0
deLKxMum9SGueh4Ib1EXPg: 4.0
z-uriBFqMdNhJBW7DsVXbQ: 2.0
jmmu7KexjmCFfiing

25IKcxpqf7uwg2sgOit2GA: 5.0
Jwq2_C7i1hl-48S0sJUGIA: 2.0
hJGbBv-tF4BSrc-8ARhTIw: 3.0
G-HFVFzgQ81EImxGZ93qYw: 4.999999181853321
P5T_vC327RVB3m8V405Oeg: 0.0
xzG2rA9sfPaeBGBTeD30vg: 5.0
wrXhhImh18nNeU29lbXIgQ: 5.0
7W_EwZnxnVA9LoPMuFn04w: 2.0
ALNiNq50YnjK1BlkMVXZZQ: 2.000000817366548
4efwXCtgEg6FAE2hp3D1Og: 2.0
M62HBfGLkdvFpMDxpDvVTA: 5.0
KCeE9YP3oZZZys8BM005Bg: 2.0
QrxqBinT8ua9VQCSFBfsHA: 1.0
JoHtbl1QSBdZRE05U73mow: 2.0
Vftp-Gc10kkBOBU7bMUULQ: 5.0
0k5N6sSv3-xNLSphYDEogQ: 2.0
zzME17DSjbBd74NnpTULNw: 2.0
LVBtfFI1HeVAY4hbfgUzXw: 1.0
h1tQkncysrpa1dWx-b1Z7Q: 3.0000209320489444
FGVWKw3C14pELBiTx2MTew: 5.0
1LjwyJiXl2B0No4LGqzlZw: 4.999999999999924
gwwd9Vhd83nZynJkDZHAbw: 4.999999661669441
MYHwIt8K_5d59s3y6BoNNQ: 0.0
Smt6yx0RnRW2WlbbY6qk2g: 2.0
KOdf7EAqK_to5kb9RoIpDQ: 5.0
xWvUUQ-tO-x9pAsG8JEnOQ: 3.0
36k2ETYAPlN9IkWLxVOpjA: 3.0
s4VJu9j_IeLCCK_1OPNHEA: 4.999999997528039
VKDik0NFXEYcWXtvAqsyKQ: 5.0
G-ohuVULZ4hHDbhA54IVXg: 2.9983143026672026
60xea5QFX2TrdfX1We8rOg: 2.000003644351543
Aq9ch0YCqkEOFJM0rK

cPeGHvbOO30YlqUkoTYLBA: 5.0
JOM6ZKgXa8JEuqlSpBBSKQ: 3.0
RQbdtwyJ2odQJd5n1hG9PQ: 1.9999999940153155
uAEuiS57FV9nrqmtpu4Wiw: 3.9458254577398946
DxYJcSy7nKFEWj_5lgiq4A: 4.0
IEuRgT7xa3W3bSLyGdBx_w: 4.9999999999972715
x1qWIA1OFd1RLG8vMtl27w: 1.9008954573148071
Ynv2_RuRGHNNeVIBS03bUA: 4.999999999971466
aQwjiO6sY5GQ5FXCMgZXCw: 4.00000023715272
z7uf22r7mpckKz3m56nhig: 5.0
9JSExLUm78MU05j2utqflg: 4.999999999888916
RMZ6LLnbit5kJN59cMlzdA: 3.999105364054646
ggBxS4kXYeWeQCyi69Accg: 2.0
xnCPHjP8k4VGqWr5ncxEsw: 3.999999999640008
8ft2WN8fA5WcPAUoyThM4g: 2.0
zvKLw8GpLzCpfhTUeqLFMw: 5.0
FiROXgFZ_9tH3AUYy2liDg: 5.0
uffKUHh3IkckowiUFpDnOQ: 5.0
G7OH384L18ccIi5nMbfVPw: 3.755651867002204
Msvolm_A25p4r4zZYbZJIg: 5.0
jsZ9quX_pvBny_IaoOsF-Q: 2.0000096137616112
yF5Sn-yEjLT-dj2C190NuA: 3.0
HBkPfmAc7mvJjRIATsd1CA: 4.999999999305973
LtSVEdXjiQF4N0anQAGrAQ: 1.9999999994698079
d1gPJtUJKaJt2788Dd1Ozg: 3.000523195364508
KZXE2cxvVnjb1YwcbAVsoA: 4.847964537190269
D0A9xp-lo5WMLJSG7bYlZg: 5.0
fdE5p2zwHQpqSHSY08DVSA: 5.0
G

tUEiqYRfIsIH00SjIb2Heg: 4.0000000472704444
7HEZa0B-MzLnDkRp8j50sw: 4.000522225853672
ygA0K8wrtKtBWvMnFmoTjw: 4.999999999985013
I5F3Zrw1ZKKal1oEa-wwvw: 1.000000000000007
zIgT5EKfKMXpgG5mFXCd8g: 4.0
CWVIyxtVOFDNcStyPk8PSQ: 3.000000000000001
auEhUsFnc99eDSS7i8x-gQ: 4.0
aXyF16LjUARK8G5roqMwUg: 3.9999894324370535
wtv5qfV7IIC4ld03sK1KxQ: 4.9999999980795184
KqEII_Ejcfmjkh0VBcJG9g: 4.0
psGDwACpn7tFmWm36865fA: 3.9999100922022848
R_GessHlfVcVPEIpNT8N_A: 4.976593154956589
0zmpfghc5E3bIO50diwjVA: 3.916333585638897
KusCIAB9JN_TTQhZC5DiVg: 4.0
WTFAWwEZd-EDLSgqbbTSSA: 3.999999989389475
N-cFtCaOLOAAL2hn9ehgYA: 1.0002593319680644
G45K87yxjqc83IGcFfit4g: 4.000000002016957
S9v-twmXQtAP8y0ATPpGPw: 3.543612627125458
dH4h3Worla2GcuZxJvnlVQ: 1.2321176739160469
5oSQX2rJO72UlxXL4A-aBA: 1.0
mL0ijmbXjuwnIAM_sSdCMg: 4.999999993935243
ElhL6G-2Y5DUuGe0KbxcTA: 3.9995747475733383
klc_A-b9FBZ9BcVO8dgR8Q: 2.999999999998237
4vvzokiVaqC2pzbJW6ZhNw: 4.200350478561534
ty56-KV-XCbeMVIdHb88Ew: 2.0000542512248916
1vtgiMokT0cK

...

-2csq9WfuWQhHQ6nlgImjA: 2.000915517078503
sVoYh1AcAB2or7ggqP48iA: 1.007220111805429
Yms9VoeU9BsVT4elLgFZ9g: 4.650511056686157
5YsNQ15o65zszWsoGhVZSA: 4.000000000322385
QbVsW68mXyOeLRWbhSCGwA: 1.0444406981883667
BXci10BbEqOak6buD81OiA: 4.999999999959105
uGol3nwbVL9tO0fgMt2P7g: 1.0007985397488248
sw3tnN6dudCWjE3ZkmN3LA: 5.0
1QmsG3vfozJg89c1-u6bJQ: 2.255109778354053
dDm6NYMgqyMf8n672LY0DQ: 4.9991699532990035
4nVNw629meAUuVvsXbnNaw: 3.9999999999793214
fvwOzLYWVgarFy7gbc8oCA: 4.000000000009462
0tjfQT4Ig8xcQC-PJSRxGg: 3.9999977430098506
DRdul-7XdH4D3_aeQIg69A: 4.990047212387553
P-7WusUE0yGDnxr1BGBZeA: 3.99999998265029
hSW-SGay9-akmhltcvotaw: 3.000003471933472
_GlmQsOTuq-RcSz3GcKDVA: 3.9999981010787184
ZUSs3lR-mnRdfO_SZxoyOw: 4.985555445466828
vO17fBWNQ87FS5yt-9hjWA: 4.882863159572367
KB1tjoJUepbl86IZvciinA: 3.974686130823168
a7DgoHGvoAYgddIR3uZGig: 2.999993100062541
uY33cuaMTsBQRqfb81O9QA: 4.99885293247447
uEnEAKXOa2iAQI7gx8XCoQ: 3.9998710445900554
soAxOvRXHU9zIeiJ-3MLMA: 2.6095704675734366


GPpaKw1jR7fzIp0-NB3cfA: 4.999999999673356
1z_CIh_s8PLEwyzEbFmpEQ: 3.999959851350035
2CQab7ebfIzyTbwWoJT0kA: 3.999737376731168
jj3wcFm9zT-DdZzU3a2bcw: 3.468516834882862
HEoE5ZRd9fbNh9Z1VzRLDw: 3.076857291369293
xv8m0zG-PWHhAlkK6vWw6w: 1.3225860594111771
c4SRXhcfESqqopq_6jgGNQ: 5.0
OqytxT_PEp8Bn8--cgaD1g: 3.9640706902464977
wxU6XNsIUnojijS1Rl_QGA: 3.0000028422341813
yHQPmwdBcYiG3xtmCNKsag: 2.5273817787574937
jNTN6o--5w6YkWFjM52O8A: 1.0000000004075742
OfAQWiyGrFQGZ2n08XYaPw: 2.0000000803351923
QS1caSQhynp20aiTWjoADQ: 2.093767324474245
GX-0YeUdz3t20TYM5_S8aA: 1.9999999950402112
o5gxUXCvgl_koe_2GxW0qQ: 4.999999999971191
GBJYIpQB-Yl9liQDC_gFLA: 4.999144780944789
4-0IWg4qdVgsN1bSAYkvaA: 2.98553618956688
m3l2uPFHFlTm5wCjZ5IiMg: 4.335836503658809
pE0iH5GyZgnZ_qa6bNt0jg: 2.7882878058585026
fene2T9N60Q7EluBmBkW5g: 4.998085673244413
O93m7Eun-58VVUIQkqIwaQ: 3.998464873250376
Q5OKU-ezWeLXmPZstw-uZw: 2.0000223887405544
7VBqqQJ9cMEqP5oAS4aoEA: 5.0
5dkjFrezuhzsdMd1IMVNNQ: 4.9999994671382435
ZasPts_S3bN

Imf1U8YMRJzOsFWre0JYbg: 4.00200792108135
l6lP4cPc_Fq576pjqGyrWQ: 3.0
udIvb-Xf9uegGCciIMPmbg: 3.8810378692365624
CLvcVkHYMTvFg1qidRh--Q: 2.0
96sE7KosAzgciMWXp0PQ3Q: 3.0
9QeagRqNj02Ve-SZDpIqoQ: 0.0
xxT8_V8hyFwD6gyhdDtYJw: 2.9423909906497707
LrL6DVzDxGUXDcRJjZOObA: 4.0
osGZealgliUIscfmy1goHw: 3.9999993763417105
nUU9kQJsPpxO2F4V6gkofQ: 3.9729594589736696
9ZgWH-15fxgiVCtLHyXocQ: 4.726011603506684
YGTyhmWf5M1oUeFDAyCwuw: 0.0
L2FcGBJRH9XFBhJ_lTkTCQ: 3.0
rwhbwznoLT4CVPOgzK_Bnw: 2.999765719895213
wSWtx0Yj6HjImKygOCqpAQ: 3.0
2_b6fClKxN_AB5Tf1h65gQ: 4.0
tXSSC1XGF2ReJ5ep9fTAuQ: 0.0
pHMV8B5sQuW4bHEdT8qzLQ: 0.0
4H1T-vJ2Ug01UtVa_U4Exw: 2.0
JAvJKSLAHQ85c-TqhiPGNA: 3.0000000000012426
lrCr75posiyiVMwO5bqDpA: 4.00034872235889
WmLW7-k_cKzvlRaAZSu_jA: 3.89730124427518
lUZB0vph3Plx3g4eBAwtHQ: 4.0
k9ke2OBtl0l3emF7rEgu0Q: 2.0000000000000013
bz0WS7HBwCvIkYC4BsB6Ng: 2.000110723107704
Qcfgsbv5JWtqHws_t8AeRg: 4.000021292339995
Fp-M4YBsIxgzrJwwCD1Jng: 4.999999430410084
qAyBTSDkKuAVXD9XZXOLmQ: 5.0
ksi1XpLxXjaCtaHaI

NzKdkFWIOP3LiWqFOksLcA: 3.990045036104613
rP3E4UrXxjUkqqdV1QKelQ: 1.000975540925636
nMSYZFr58J7p-QdjNVoBgA: 4.939707235283724
9N18Vk_A5Ah0gGFeXFeNXw: 1.0
tXLrxE87NCnbgpWWKKwx_A: 3.0000000000000373
i1fonJilgfTFBkRS0IEN3w: 1.0
wr7RnGCx-P4WZPv7Yo3yEg: 5.0
f-Hqo35jS187FWlM55quwg: 1.020473102598226
AhNHXwqjYsw11ycYq4SlXg: 4.999999999956229
R_pPt96xkBgvdFvoRz567w: 2.0000237330051718
MZ1k9K9xmdkFSKQG_nIG8A: 1.033887542504085
KcdzSfKP7amCWR_cScjgig: 1.0000000000000002
8ZYdj2MUuBapFeVqiPeL8w: 4.999997926767644
5mGRyNSFLZjj-t6EouqnuQ: 1.0000000000000169
2R1nSYX9vpzNzGXa3ZuowA: 2.000000000198007
W0liwsn_kan6brqbwsrrTA: 4.957692520445079
QBcaKhk29ZysU4GPX1m2KQ: 4.999999503334042
sRo1Ff22VfWFMsmizSgbSQ: 4.283835966644842
cANxJlaBpdJczEXiI7u4ow: 2.9999999841112928
8YYhC0vqsvnu90OLb8EsLA: 4.000128252821587
pHQmwlQfkNIhtMtRSHt-QA: 4.955253911845985
3PcFLGFMC2bqJMQkAzTlMg: 4.426082106857592
660RvotWl6r414B_dlPssA: 1.999999999990095
jXz3L5uEMWtnvSfNJhhn_g: 1.0
z5tpCHltEwIj-Gd2E6cU7g: 3.000054284165179
Q

hWYaHDwfb1uu-QiWjb30sw: 1.0
mykvZVJo5UC4FYgVFpHu8Q: 2.0002236660041754
VIwwkatF3Az4yNgdmuboXA: 3.000815076118607
LT4OW-BLt_x7kGBJ0aD_Jw: 2.0
O1674PAQUSdm60ITa0F5JQ: 2.592371694830749
kIjkJ_4ya10VUVaXScMmog: 3.0490371818749344
Ha3pVYlX7Wy-NNDL9zoelA: 2.686700482790086
vwW3aSjzHgPPKOZ3Q7UDiw: 2.999999963657611
2t8v2xcR-LF8PcezkCo0QA: 4.999999689863801
cURR4hoe26jlWIexzUN58A: 2.0001889589862034
qSnSkv8AWBrF2aUnvbJfEw: 2.0000469928038798
yVVbNZhOc-z3Elir1ENiPw: 5.0
TEHvYrURlau7-wTwELYZ_Q: 2.965411610541138
pOFoPUhOsDb88DwgrKs2hA: 1.0000014346669566
R-HTxqSy8s5w0ZUaFP48jA: 4.000001336029119
-hxV4RbKqakk5uTB-DVIIw: 1.002407862617948
FqKmtB_jZPYDM9Bzx4PZEQ: 5.0
Cq_0UW-kVCb_jQkLJSRQrw: 4.170429126620112
K1MjgzPsc2WHmJMH_aQQ8A: 4.999950215748444
lxnhCO-lBalLySOGiyfgWQ: 4.999999999999971
wwScG_tJMGS28A-GBuFNqg: 3.0000000000000018
DM6BNouJm72Q2gBPIGQYfQ: 2.0006895970408802
JT7jtObBuQh7pMWCjQ2dvQ: 3.999997264477871
ewYt_HlUrnADfzlcbDHYTg: 5.0
GxQiPHni7cvZnX-m1AMngQ: 2.0005953293910146
wJhhQ6afMeNy

In [10]:
#dataset.RankBigramReviews(save=True, weightedMean = True)